# Objective

This notebooks aims to ingest, clean and persist the data

# Import essentials

In [1]:
import pandas as pd
import json
from pathlib import Path
import sys

print(sys.version)

3.10.6 (main, Aug 10 2022, 11:40:04) [GCC 11.3.0]


# Data ingestion

In [2]:
# check schema of data

with open("../data/raw/dataset.json", "r") as f:
    data = json.load(f)

In [3]:
print(len(data))

160000


In [4]:
print(json.dumps(data[0], indent=4))

{
    "browser": "Chrome",
    "os": "Windows 8",
    "locale": "de-DE",
    "user_id": 164,
    "gender": "m",
    "location": "Canada/Toronto",
    "sites": [
        {
            "site": "lenta.ru",
            "length": 296
        },
        {
            "site": "lenta.ru",
            "length": 69
        },
        {
            "site": "vk.com",
            "length": 94
        },
        {
            "site": "lenta.ru",
            "length": 129
        },
        {
            "site": "wikipedia.org",
            "length": 70
        },
        {
            "site": "instagram.com",
            "length": 120
        },
        {
            "site": "msn.com",
            "length": 54
        },
        {
            "site": "amazon.com",
            "length": 213
        },
        {
            "site": "outlook.com",
            "length": 140
        },
        {
            "site": "verisign.com",
            "length": 166
        },
        {
            "site": "google

# Data prep - 1

## Prepare a table for high level session information

In [4]:
# unnest and normalize
def get_records():
    for ele in data:
        filtered = ele.copy()
        filtered.pop("sites")
        yield filtered

In [6]:
df_session = pd.DataFrame.from_records(get_records())

In [7]:
df_session.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160000 entries, 0 to 159999
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   browser   160000 non-null  object
 1   os        160000 non-null  object
 2   locale    160000 non-null  object
 3   user_id   160000 non-null  int64 
 4   gender    160000 non-null  object
 5   location  160000 non-null  object
 6   time      160000 non-null  object
 7   date      160000 non-null  object
dtypes: int64(1), object(7)
memory usage: 9.8+ MB


So there are no null records in the raw data. However, we could still cast certain columns in the right data type

In [8]:
# ensure that there are no duplicate records
assert len(df_session) == len(df_session.drop_duplicates())

In [9]:
df_session["user_id"] = df_session["user_id"].astype(int)

In [10]:
df_session["timestamp"] = pd.to_datetime(df_session["date"]+"T"+df_session["time"], format="%Y-%m-%dT%H:%M:%S")

In [11]:
df_session.drop(columns=["date", "time"], inplace=True)

In [12]:
df_session = df_session.reset_index().rename(columns={"index":"session_id"})

In [13]:
df_session.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160000 entries, 0 to 159999
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   session_id  160000 non-null  int64         
 1   browser     160000 non-null  object        
 2   os          160000 non-null  object        
 3   locale      160000 non-null  object        
 4   user_id     160000 non-null  int64         
 5   gender      160000 non-null  object        
 6   location    160000 non-null  object        
 7   timestamp   160000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 9.8+ MB


## Prepare a table for site-level session information

In [5]:
def get_sites():
    for idx, ele in enumerate(data):
        df_ = pd.DataFrame.from_dict(ele.get("sites"))
        df_["session_id"] = idx
        yield df_

In [15]:
df_sites = pd.concat(get_sites()).reset_index(drop=True)

In [16]:
df_sites.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2065873 entries, 0 to 2065872
Data columns (total 3 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   site        object
 1   length      int64 
 2   session_id  int64 
dtypes: int64(2), object(1)
memory usage: 47.3+ MB


In [17]:
df_sites.sample(5)

,site,length,session_id
586538,facebook.net,124,45447
1174494,icloud.com,108,90997
38055,mail.google.com,65,2949
1994617,amazon.com,43,154474
503123,outlook.com,140,38996


# Save the data to disk

In [25]:
path_to_folder = Path.cwd().joinpath("../data/prep_1")

In [26]:
path_to_folder.mkdir(exist_ok=True)

In [27]:
df_session.to_feather(path_to_folder.joinpath("session.feather"))

In [28]:
df_sites.to_feather(path_to_folder.joinpath("sites.feather"))